# KMeans

In [2]:
import cluster_data_pca
from cluster_data_pca import run_clustering, normalize_data, unnormalize
import numpy as np
import pandas as pd
import os
from kmeans import k_means
from cluster_plotter import ClusterPlotter
from clustering_utils_pca import ClusterData
import cluster_plotter
import high_dim_analysis
import seaborn as sns
import matplotlib.pyplot as plt

standard_year_ranges = {
    f"{start}-{start + 3}": np.arange(start, start + 4)
    for start in [2002, 2006, 2010, 2014, 2019]
}
standard_year_ranges["2019-2023"] = np.arange(2019, 2024)  # Include 2023
running_ranges = cluster_data_pca.generate_running_year_ranges(2002, 2023, 4)

binned_data = cluster_data_pca.bin_data_for_clustering(running_ranges, print_res=False)

k_values = [5, 6, 7]

results_per_year_range = {}

# Output directory for plots
plot_dir = "Images\\k_means_running_bins_pca"
os.makedirs(plot_dir, exist_ok=True)
cluster_plotter.clear_directory(plot_dir)

for cluster_data, year_range in binned_data:  
    print(f"\nRunning K-Means for Year Range: {year_range}")

    data_array = np.array([cluster_data.ecc, cluster_data.mag_obj, cluster_data.sem_maj, cluster_data.diameter, cluster_data.inc, cluster_data.raan]).T
    feature_names = ["Eccentricity e", "Object magnitude [mag]", "Semi major axis [km]", "Diameter [m]", "Inclination [°]", "RAAN [°]"]

    normalized_data, data_min, data_max = normalize_data(data_array)

    k_values = [3, 4, 5, 6, 7]

    results = []

    for k in k_values:
        result_kmeans, time_kmeans, n_clusters_kmeans, points_per_cluster_kmeans, metrics_kmeans = run_clustering(
            k_means, f"K-means (k={k})", normalized_data, data_min, data_max, k, init='kmeans++'
        )

        unnormalized_data, cluster_centers = unnormalize(
            result_kmeans.data, result_kmeans.cluster_centers, data_min, data_max
        )

        results.append({
            "Year Range": year_range,
            "k": k,
            "Runtime (s)": f"{time_kmeans:.3f}",
            "Clusters": n_clusters_kmeans,
            "Points per Cluster": points_per_cluster_kmeans,
            "Davies-Bouldin": f"{metrics_kmeans[0]:.3f}" if metrics_kmeans and isinstance(metrics_kmeans[0], (int, float)) else None,
            "Calinski-Harabasz": f"{metrics_kmeans[1]:.3f}" if metrics_kmeans and isinstance(metrics_kmeans[1], (int, float)) else None,
            "Dunn Index": f"{metrics_kmeans[2]:.3f}" if metrics_kmeans and isinstance(metrics_kmeans[2], (int, float)) else None,
            "Silhouette Score": f"{metrics_kmeans[3]:.3f}" if metrics_kmeans and isinstance(metrics_kmeans[3], (int, float)) else None
        })

        labels = result_kmeans.labels
        """high_dim_analysis.plot_co_membership(
            labels,
            output_folder=plot_dir,
            filename=f"co_membership_{year_range}_k{k}.png"
        )"""
        high_dim_analysis.plot_correlation_heatmap(
            cluster_data,
            output_folder=plot_dir,
            filename=f"corr_{year_range}_k{k}.png"
        )
        high_dim_analysis.plot_mutual_information(
            cluster_data,
            labels,
            output_folder=plot_dir,
            filename=f"mi_{year_range}_k{k}.png"
        )

        from sklearn.feature_selection import f_classif
        X = np.array([cluster_data.ecc, cluster_data.mag_obj, cluster_data.sem_maj,
              cluster_data.diameter, cluster_data.inc, cluster_data.raan]).T
        valid = labels != -1  # Exclude noise points (usually not present for kmeans)
        if np.any(valid) and len(np.unique(labels[valid])) > 1:
            f_vals, _ = f_classif(X[valid], labels[valid])
            mean_f = np.mean(f_vals)
        else:
            mean_f = 0.0  # Avoid invalid or single-cluster cases

        results[-1]["Mean ANOVA F"] = f"{mean_f:.3f}"

    df = pd.DataFrame(results)
    df["Mean ANOVA F"] = pd.to_numeric(df["Mean ANOVA F"], errors="coerce")
    df_sorted = df.sort_values(by="Mean ANOVA F", ascending=False)
    best_params = df_sorted.iloc[0]
    best_k = best_params["k"]

    # Re-run clustering with best parameters to get labels
    best_result, _, _, _, _ = run_clustering(
        k_means, f"K-means (k={best_k})", normalized_data, data_min, data_max, int(best_k), init='kmeans++'
    )
    best_labels = best_result.labels

    # Plot pairplot colored by cluster labels and save to plot_dir
    df_plot = pd.DataFrame(data_array, columns=feature_names)
    df_plot['cluster'] = best_labels.astype(str)  # cluster labels as strings for hue

    pairplot = sns.pairplot(df_plot, hue='cluster', diag_kind='kde', plot_kws={'alpha': 0.6, 's': 8})
    pairplot.fig.suptitle(f"K-Means Clusters for Year Range {year_range}", y=1.02)
    pairplot.savefig(os.path.join(plot_dir, f"pairplot_{year_range}_k{best_k}.png"))
    plt.close(pairplot.fig)

    # Plot ANOVA F-values only for best result
    high_dim_analysis.plot_anova_f_values(
        cluster_data,
        best_labels,
        output_folder=plot_dir,
        filename=f"anova_best_{year_range}_k{best_k}.png",
        title=f"ANOVA F-values (Best KMeans Parameters), k = {int(best_k)}"
    )
    print("Best KMeans parameters based on ANOVA F-value:")
    print(best_params)

    display(df)


Running K-Means for Year Range: 2002-2005
Runtime for k_means: 0.007735 seconds
Runtime for k_means: 0.018622 seconds
Runtime for k_means: 0.047746 seconds
Runtime for k_means: 0.091677 seconds
Runtime for k_means: 0.141947 seconds
Runtime for k_means: 0.047482 seconds


c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:43: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=f_vals, y=features, palette="crest", legend = False)


Best KMeans parameters based on ANOVA F-value:
Year Range                              2002-2005
k                                               3
Runtime (s)                                 0.008
Clusters                                        3
Points per Cluster    {0: 4362, 1: 1359, 2: 2316}
Davies-Bouldin                               None
Calinski-Harabasz                            None
Dunn Index                                   None
Silhouette Score                             None
Mean ANOVA F                            28742.022
Name: 0, dtype: object


,Year Range,k,Runtime (s),Clusters,Points per Cluster,Davies-Bouldin,Calinski-Harabasz,Dunn Index,Silhouette Score,Mean ANOVA F
0,2002-2005,3,0.008,3,"{0: 4362, 1: 1359, 2: 2316}",None,None,None,None,28742.022
1,2002-2005,4,0.019,4,"{0: 1208, 1: 3155, 2: 1359, 3: 2315}",None,None,None,None,19750.659
2,2002-2005,5,0.048,5,"{0: 1495, 1: 514, 2: 1359, 3: 2354, 4: 2315}",None,None,None,None,15038.282
3,2002-2005,6,0.092,6,"{0: 735, 1: 3154, 2: 499, 3: 859, 4: 1208, 5: ...",None,None,None,None,12884.282
4,2002-2005,7,0.142,7,"{0: 1578, 1: 180, 2: 3041, 3: 739, 4: 495, 5: ...",None,None,None,None,10986.996



Running K-Means for Year Range: 2003-2006
Runtime for k_means: 0.051761 seconds
Runtime for k_means: 0.038968 seconds
Runtime for k_means: 0.070328 seconds
Runtime for k_means: 0.058903 seconds
Runtime for k_means: 0.137317 seconds
Runtime for k_means: 0.024183 seconds
Best KMeans parameters based on ANOVA F-value:
Year Range                              2003-2006
k                                               3
Runtime (s)                                 0.052
Clusters                                        3
Points per Cluster    {0: 1335, 1: 3874, 2: 3670}
Davies-Bouldin                               None
Calinski-Harabasz                            None
Dunn Index                                   None
Silhouette Score                             None
Mean ANOVA F                            28907.517
Name: 0, dtype: object


...

,Year Range,k,Runtime (s),Clusters,Points per Cluster,Davies-Bouldin,Calinski-Harabasz,Dunn Index,Silhouette Score,Mean ANOVA F
0,2019-2022,3,0.033,3,"{0: 1717, 1: 1317, 2: 575}",None,None,None,None,5229.101
1,2019-2022,4,0.018,4,"{0: 1720, 1: 571, 2: 515, 3: 803}",None,None,None,None,4539.420
2,2019-2022,5,0.016,5,"{0: 1302, 1: 481, 2: 966, 3: 352, 4: 508}",None,None,None,None,3305.395
3,2019-2022,6,0.064,6,"{0: 170, 1: 414, 2: 2262, 3: 300, 4: 350, 5: 113}",None,None,None,None,4634.594
4,2019-2022,7,0.067,7,"{0: 572, 1: 500, 2: 759, 3: 261, 4: 954, 5: 30...",None,None,None,None,2432.329



Running K-Means for Year Range: 2020-2023
Runtime for k_means: 0.019859 seconds
Runtime for k_means: 0.030695 seconds
Runtime for k_means: 0.047903 seconds
Runtime for k_means: 0.158988 seconds
Runtime for k_means: 0.131779 seconds
Runtime for k_means: 0.067818 seconds


c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:43: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=f_vals, y=features, palette="crest", legend = False)


Best KMeans parameters based on ANOVA F-value:
Year Range                                             2020-2023
k                                                              5
Runtime (s)                                                0.048
Clusters                                                       5
Points per Cluster    {0: 1545, 1: 586, 2: 613, 3: 1001, 4: 627}
Davies-Bouldin                                              None
Calinski-Harabasz                                           None
Dunn Index                                                  None
Silhouette Score                                            None
Mean ANOVA F                                            9320.676
Name: 2, dtype: object


,Year Range,k,Runtime (s),Clusters,Points per Cluster,Davies-Bouldin,Calinski-Harabasz,Dunn Index,Silhouette Score,Mean ANOVA F
0,2020-2023,3,0.020,3,"{0: 746, 1: 2054, 2: 1572}",None,None,None,None,6472.985
1,2020-2023,4,0.031,4,"{0: 621, 1: 1547, 2: 632, 3: 1572}",None,None,None,None,4481.522
2,2020-2023,5,0.048,5,"{0: 1545, 1: 586, 2: 613, 3: 1001, 4: 627}",None,None,None,None,9320.676
3,2020-2023,6,0.159,6,"{0: 862, 1: 615, 2: 1547, 3: 402, 4: 315, 5: 631}",None,None,None,None,3456.960
4,2020-2023,7,0.132,7,"{0: 1547, 1: 486, 2: 609, 3: 369, 4: 520, 5: 6...",None,None,None,None,5173.775


# KMeans
The features used by the clustering are determined by applying pca on the raw dataset.   
After the clustering, we reapply the PCA and plot the clusters in the PC1 / PC2 plane and make the pairplot with the clustered data. 

In [4]:
import cluster_data_pca
from cluster_data_pca import run_clustering, normalize_data, unnormalize
import numpy as np
import pandas as pd
import os
from kmeans import k_means
from cluster_plotter import ClusterPlotter
from clustering_utils_pca import ClusterData
import cluster_plotter
import high_dim_analysis
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

standard_year_ranges = {
    f"{start}-{start + 3}": np.arange(start, start + 4)
    for start in [2002, 2006, 2010, 2014, 2019]
}
standard_year_ranges["2019-2023"] = np.arange(2019, 2024)
running_ranges = cluster_data_pca.generate_running_year_ranges(2002, 2023, 4)

binned_data = cluster_data_pca.bin_data_for_clustering(running_ranges, print_res=False)

k_values = [5, 6, 7]

results_per_year_range = {}

plot_dir = "Images\\k_means_running_bins_pcabased_pcaevaluated"
os.makedirs(plot_dir, exist_ok=True)
cluster_plotter.clear_directory(plot_dir)

for cluster_data, year_range in binned_data:
    print(f"\nRunning K-Means for Year Range: {year_range}")

    data_array = np.array([cluster_data.ecc, cluster_data.mag_obj, cluster_data.sem_maj, cluster_data.diameter, cluster_data.inc, cluster_data.raan]).T
    feature_names = ["Eccentricity e", "Object magnitude [mag]", "Semi major axis [km]", "Diameter [m]", "Inclination [°]", "RAAN [°]"]

    normalized_data, data_min, data_max = normalize_data(data_array)

    k_values = [3, 4, 5, 6, 7]
    results = []

    for k in k_values:
        result_kmeans, time_kmeans, n_clusters_kmeans, points_per_cluster_kmeans, metrics_kmeans = run_clustering(
            k_means, f"K-means (k={k})", normalized_data, data_min, data_max, k, init='kmeans++'
        )

        unnormalized_data, cluster_centers = unnormalize(
            result_kmeans.data, result_kmeans.cluster_centers, data_min, data_max
        )

        results.append({
            "Year Range": year_range,
            "k": k,
            "Runtime (s)": f"{time_kmeans:.3f}",
            "Clusters": n_clusters_kmeans,
            "Points per Cluster": points_per_cluster_kmeans,
            "Davies-Bouldin": f"{metrics_kmeans[0]:.3f}" if metrics_kmeans and isinstance(metrics_kmeans[0], (int, float)) else None,
            "Calinski-Harabasz": f"{metrics_kmeans[1]:.3f}" if metrics_kmeans and isinstance(metrics_kmeans[1], (int, float)) else None,
            "Dunn Index": f"{metrics_kmeans[2]:.3f}" if metrics_kmeans and isinstance(metrics_kmeans[2], (int, float)) else None,
            "Silhouette Score": f"{metrics_kmeans[3]:.3f}" if metrics_kmeans and isinstance(metrics_kmeans[3], (int, float)) else None
        })

        labels = result_kmeans.labels

        high_dim_analysis.plot_correlation_heatmap(
            cluster_data,
            output_folder=plot_dir,
            filename=f"corr_{year_range}_k{k}.png"
        )
        high_dim_analysis.plot_mutual_information(
            cluster_data,
            labels,
            output_folder=plot_dir,
            filename=f"mi_{year_range}_k{k}.png"
        )

        from sklearn.feature_selection import f_classif
        X = data_array
        valid = labels != -1
        if np.any(valid) and len(np.unique(labels[valid])) > 1:
            f_vals, _ = f_classif(X[valid], labels[valid])
            mean_f = np.mean(f_vals)
        else:
            mean_f = 0.0

        results[-1]["Mean ANOVA F"] = f"{mean_f:.3f}"

    df = pd.DataFrame(results)
    df["Mean ANOVA F"] = pd.to_numeric(df["Mean ANOVA F"], errors="coerce")
    df_sorted = df.sort_values(by="Mean ANOVA F", ascending=False)
    best_params = df_sorted.iloc[0]
    best_k = best_params["k"]

    best_result, _, _, _, _ = run_clustering(
        k_means, f"K-means (k={best_k})", normalized_data, data_min, data_max, int(best_k), init='kmeans++'
    )
    best_labels = best_result.labels

    df_plot = pd.DataFrame(data_array, columns=feature_names)
    df_plot['cluster'] = best_labels.astype(str)

    pairplot = sns.pairplot(df_plot, hue='cluster', diag_kind='kde', plot_kws={'alpha': 0.6, 's': 8})
    pairplot.fig.suptitle(f"K-Means Clusters for Year Range {year_range}", y=1.02)
    pairplot.savefig(os.path.join(plot_dir, f"pairplot_{year_range}_k{best_k}.png"))
    plt.close(pairplot.fig)

    # PCA Plot colored by clusters
    scaler = StandardScaler()
    scaled = scaler.fit_transform(data_array)
    pca = PCA(n_components=2)
    components = pca.fit_transform(scaled)
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(components[:, 0], components[:, 1], c=best_labels, cmap='tab10', s=8, alpha=0.7)
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.title(f'PCA: PC1 vs PC2 ({year_range})')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(plot_dir, f"pca_pc1_pc2_{year_range}_k{best_k}.png"))
    plt.close()

    high_dim_analysis.plot_anova_f_values(
        cluster_data,
        best_labels,
        output_folder=plot_dir,
        filename=f"anova_best_{year_range}_k{best_k}.png",
        title=f"ANOVA F-values (Best KMeans Parameters), k = {int(best_k)}"
    )
    print("Best KMeans parameters based on ANOVA F-value:")
    print(best_params)

    display(df)


Running K-Means for Year Range: 2002-2005
Runtime for k_means: 0.023695 seconds
Runtime for k_means: 0.033191 seconds
Runtime for k_means: 0.242683 seconds
Runtime for k_means: 0.200656 seconds
Runtime for k_means: 0.411942 seconds
Runtime for k_means: 0.041855 seconds
Best KMeans parameters based on ANOVA F-value:
Year Range                              2002-2005
k                                               3
Runtime (s)                                 0.024
Clusters                                        3
Points per Cluster    {0: 3149, 1: 1216, 2: 3672}
Davies-Bouldin                               None
Calinski-Harabasz                            None
Dunn Index                                   None
Silhouette Score                             None
Mean ANOVA F                            27762.699
Name: 0, dtype: object


c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:43: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=f_vals, y=features, palette="crest", legend = False)


,Year Range,k,Runtime (s),Clusters,Points per Cluster,Davies-Bouldin,Calinski-Harabasz,Dunn Index,Silhouette Score,Mean ANOVA F
0,2002-2005,3,0.024,3,"{0: 3149, 1: 1216, 2: 3672}",None,None,None,None,27762.699
1,2002-2005,4,0.033,4,"{0: 2316, 1: 3155, 2: 1208, 3: 1358}",None,None,None,None,19752.603
2,2002-2005,5,0.243,5,"{0: 1234, 1: 3151, 2: 1211, 3: 772, 4: 1669}",None,None,None,None,15254.666
3,2002-2005,6,0.201,6,"{0: 934, 1: 1225, 2: 3146, 3: 1216, 4: 821, 5:...",None,None,None,None,12848.121
4,2002-2005,7,0.412,7,"{0: 4360, 1: 967, 2: 455, 3: 465, 4: 450, 5: 6...",None,None,None,None,12679.857



Running K-Means for Year Range: 2003-2006
Runtime for k_means: 0.028220 seconds
Runtime for k_means: 0.047288 seconds
Runtime for k_means: 0.066481 seconds
Runtime for k_means: 0.116948 seconds
Runtime for k_means: 0.150879 seconds
Runtime for k_means: 0.015888 seconds


c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:43: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=f_vals, y=features, palette="crest", legend = False)


...

,Year Range,k,Runtime (s),Clusters,Points per Cluster,Davies-Bouldin,Calinski-Harabasz,Dunn Index,Silhouette Score,Mean ANOVA F
0,2019-2022,3,0.033,3,"{0: 2286, 1: 352, 2: 971}",None,None,None,None,5694.683
1,2019-2022,4,0.000,4,"{0: 482, 1: 1302, 2: 509, 3: 1316}",None,None,None,None,3616.997
2,2019-2022,5,0.003,5,"{0: 635, 1: 1707, 2: 446, 3: 563, 4: 258}",None,None,None,None,6002.438
3,2019-2022,6,0.016,6,"{0: 1307, 1: 495, 2: 330, 3: 501, 4: 469, 5: 507}",None,None,None,None,5938.169
4,2019-2022,7,0.033,7,"{0: 1306, 1: 425, 2: 188, 3: 497, 4: 468, 5: 2...",None,None,None,None,4182.057



Running K-Means for Year Range: 2020-2023
Runtime for k_means: 0.016548 seconds
Runtime for k_means: 0.017481 seconds
Runtime for k_means: 0.033439 seconds
Runtime for k_means: 0.018434 seconds
Runtime for k_means: 0.040402 seconds
Runtime for k_means: 0.017027 seconds


c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:43: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=f_vals, y=features, palette="crest", legend = False)


Best KMeans parameters based on ANOVA F-value:
Year Range                             2020-2023
k                                              3
Runtime (s)                                0.017
Clusters                                       3
Points per Cluster    {0: 2775, 1: 1006, 2: 591}
Davies-Bouldin                              None
Calinski-Harabasz                           None
Dunn Index                                  None
Silhouette Score                            None
Mean ANOVA F                           17473.893
Name: 0, dtype: object


,Year Range,k,Runtime (s),Clusters,Points per Cluster,Davies-Bouldin,Calinski-Harabasz,Dunn Index,Silhouette Score,Mean ANOVA F
0,2020-2023,3,0.017,3,"{0: 2775, 1: 1006, 2: 591}",None,None,None,None,17473.893
1,2020-2023,4,0.017,4,"{0: 1078, 1: 744, 2: 497, 3: 2053}",None,None,None,None,5112.742
2,2020-2023,5,0.033,5,"{0: 581, 1: 2040, 2: 412, 3: 741, 4: 598}",None,None,None,None,9115.729
3,2020-2023,6,0.018,6,"{0: 875, 1: 631, 2: 314, 3: 615, 4: 1547, 5: 390}",None,None,None,None,3447.406
4,2020-2023,7,0.040,7,"{0: 512, 1: 613, 2: 626, 3: 1544, 4: 210, 5: 4...",None,None,None,None,5204.487


# DBSCAN

In [3]:
import cluster_data_pca
from cluster_data_pca import run_clustering, normalize_data, unnormalize
import numpy as np
import pandas as pd
import os
from kmeans import k_means
from cluster_plotter import ClusterPlotter
from clustering_utils_pca import ClusterData
import cluster_plotter
from DBSCAN import dbscan_clustering
import high_dim_analysis
import seaborn as sns
import matplotlib.pyplot as plt

# Define year ranges
standard_year_ranges = {
    f"{start}-{start + 3}": np.arange(start, start + 4)
    for start in [2002, 2006, 2010, 2014, 2019]
}
standard_year_ranges["2019-2023"] = np.arange(2019, 2024)  # Include 2023
running_ranges = cluster_data_pca.generate_running_year_ranges(2002, 2023, 4)

binned_data = cluster_data_pca.bin_data_for_clustering(running_ranges, print_res=False)

results_per_year_range = {}

# Output directory for plots
plot_dir = r"Images\dbscan_tests_running_bins_pca"
os.makedirs(plot_dir, exist_ok=True)
cluster_plotter.clear_directory(plot_dir)

for cluster_data, year_range in binned_data:  
    print(f"\nRunning DBSCAN for Year Range: {year_range}")
    data_array = np.array([cluster_data.ecc, cluster_data.mag_obj, cluster_data.sem_maj, cluster_data.diameter, cluster_data.inc, cluster_data.raan]).T
    feature_names = ["Eccentricity e", "Object magnitude [mag]", "Semi major axis [km]", "Diameter [m]", "Inclination [°]", "RAAN [°]"]

    normalized_data, data_min, data_max = normalize_data(data_array)

    # DBSCAN parameter ranges
    eps_values = [0.02, 0.01, 0.015]
    min_samples_values = [10, 15, 25, 30]

    results = []

    for eps in eps_values:
        for min_samples in min_samples_values:
            result_dbscan, time_dbscan, n_clusters_dbscan, points_per_cluster_dbscan, metrics_dbscan = run_clustering(
                dbscan_clustering, "DBSCAN", normalized_data, data_min, data_max, eps=eps, min_samples=min_samples
            )

            unnormalized_data, _ = unnormalize(
                result_dbscan.data, None, data_min, data_max
            )
            # Count noise points
            noise_points = np.sum(result_dbscan.labels == -1)

            results.append({
                "Year Range": year_range,
                "eps": eps,
                "min_samples": min_samples,
                "Runtime (s)": f"{time_dbscan:.3f}",
                "Clusters": n_clusters_dbscan,
                "Points per Cluster": points_per_cluster_dbscan,
                "Noise Points": noise_points,
                "Davies-Bouldin": f"{metrics_dbscan[0]:.3f}" if metrics_dbscan and isinstance(metrics_dbscan[0], (int, float)) else None,
                "Calinski-Harabasz": f"{metrics_dbscan[1]:.3f}" if metrics_dbscan and isinstance(metrics_dbscan[1], (int, float)) else None
            })


            labels = result_dbscan.labels
            """high_dim_analysis.plot_co_membership(
                labels,
                output_folder=plot_dir,
                filename=f"co_membership_{year_range}_eps{eps}_minsamples_{min_samples}.png"
            )"""
            high_dim_analysis.plot_correlation_heatmap(
                cluster_data,
                output_folder=plot_dir,
                filename=f"corr_{year_range}_eps{eps}_minsamples_{min_samples}.png"
            )
            high_dim_analysis.plot_mutual_information(
                cluster_data,
                labels,
                output_folder=plot_dir,
                filename=f"mi_{year_range}_eps{eps}_minsamples_{min_samples}.png"
            )
            
        from sklearn.feature_selection import f_classif
        X = np.array([cluster_data.ecc, cluster_data.mag_obj, cluster_data.sem_maj,
              cluster_data.diameter, cluster_data.inc, cluster_data.raan]).T
        valid = labels != -1  # Exclude noise points
        if np.any(valid) and len(np.unique(labels[valid])) > 1:
            f_vals, _ = f_classif(X[valid], labels[valid])
            weights = np.var(X[valid], axis=0)
            weighted_mean_f = np.average(f_vals, weights=weights)
        else:
            weighted_mean_f = 0.0  # Avoid invalid or single-cluster cases

        results[-1]["Weighted Mean ANOVA F"] = f"{weighted_mean_f:.3f}"

    df = pd.DataFrame(results)

    df["Weighted Mean ANOVA F"] = pd.to_numeric(df["Weighted Mean ANOVA F"], errors="coerce")
    df_sorted = df.sort_values(by="Weighted Mean ANOVA F", ascending=False)
    best_params = df_sorted.iloc[0]
    best_eps = best_params["eps"]
    best_min_samples = best_params["min_samples"]

    # Re-run clustering with best parameters to get labels
    best_result, _, _, _, _ = run_clustering(
        dbscan_clustering, "DBSCAN", normalized_data, data_min, data_max,
        eps=best_eps, min_samples=best_min_samples
    )
    best_labels = best_result.labels

    df_plot = pd.DataFrame(data_array, columns=feature_names)
    df_plot['cluster'] = best_labels.astype(str)

    pairplot = sns.pairplot(df_plot, hue='cluster', diag_kind='kde', plot_kws={'alpha':0.6, 's':40})
    pairplot.fig.suptitle(f"DBSCAN Clusters for Year Range {year_range}", y=1.02)

    # Save the figure to the plot directory
    pairplot.savefig(os.path.join(plot_dir, f"pairplot_{year_range}_eps{best_eps}_minsamples_{best_min_samples}.png"))

    plt.close(pairplot.fig)  # Close the figure to free memory


    # Plot ANOVA F-values only for best result
    high_dim_analysis.plot_anova_f_values(
        cluster_data,
        best_labels,
        output_folder=plot_dir,
        filename=f"anova_best_{year_range}_eps{best_eps}_minsamples_{best_min_samples}.png",
        title=f"ANOVA F-values (Best DBSCAN Parameters), eps = {float(best_eps)}, minsamples = {float(best_min_samples)}"
    )

    print("Best DBSCAN parameters based on ANOVA F-value:")
    print(best_params)
    display(df)



Running DBSCAN for Year Range: 2002-2005
Runtime for dbscan_clustering: 0.160578 seconds
Runtime for dbscan_clustering: 0.160060 seconds
Runtime for dbscan_clustering: 0.174719 seconds
Runtime for dbscan_clustering: 0.159367 seconds
Runtime for dbscan_clustering: 0.129251 seconds
Runtime for dbscan_clustering: 0.165635 seconds
Runtime for dbscan_clustering: 0.124753 seconds
Runtime for dbscan_clustering: 0.139642 seconds
Runtime for dbscan_clustering: 0.158799 seconds
Runtime for dbscan_clustering: 0.161056 seconds
Runtime for dbscan_clustering: 0.127797 seconds
Runtime for dbscan_clustering: 0.142746 seconds
Runtime for dbscan_clustering: 0.174946 seconds


c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:43: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=f_vals, y=features, palette="crest", legend = False)


Best DBSCAN parameters based on ANOVA F-value:
Year Range                                                     2002-2005
eps                                                                 0.02
min_samples                                                           30
Runtime (s)                                                        0.159
Clusters                                                               6
Points per Cluster       {0: 122, 1: 314, 2: 96, 3: 75, 4: 46, -1: 7384}
Noise Points                                                        7384
Davies-Bouldin                                                      None
Calinski-Harabasz                                                   None
Weighted Mean ANOVA F                                            112.751
Name: 3, dtype: object


,Year Range,eps,min_samples,Runtime (s),Clusters,Points per Cluster,Noise Points,Davies-Bouldin,Calinski-Harabasz,Weighted Mean ANOVA F
0,2002-2005,0.020,10,0.161,28,"{0: 21, 1: 28, 2: 37, 3: 1009, 4: 17, 5: 141, ...",6159,None,None,NaN
1,2002-2005,0.020,15,0.160,15,"{0: 15, 1: 21, 2: 210, 3: 717, 4: 13, 5: 25, 6...",6625,None,None,NaN
2,2002-2005,0.020,25,0.175,6,"{0: 557, 1: 101, 2: 66, 3: 64, 4: 40, -1: 7209}",7209,None,None,NaN
3,2002-2005,0.020,30,0.159,6,"{0: 122, 1: 314, 2: 96, 3: 75, 4: 46, -1: 7384}",7384,None,None,112.751
4,2002-2005,0.010,10,0.129,3,"{0: 33, 1: 22, -1: 7982}",7982,None,None,NaN
5,2002-2005,0.010,15,0.166,2,"{0: 31, -1: 8006}",8006,None,None,NaN
6,2002-2005,0.010,25,0.125,1,{-1: 8037},8037,None,None,NaN
7,2002-2005,0.010,30,0.140,1,{-1: 8037},8037,None,None,0.000
8,2002-2005,0.015,10,0.159,25,"{0: 11, 1: 16, 2: 292, 3: 94, 4: 44, 5: 80, 6:...",7115,None,None,NaN
9,2002-2005,0.015,15,0.161,8,"{0: 75, 1: 232, 2: 71, 3: 15, 4: 17, 5: 16, 6:...",7587,None,None,NaN



Running DBSCAN for Year Range: 2003-2006
Runtime for dbscan_clustering: 0.241416 seconds
Runtime for dbscan_clustering: 0.202453 seconds
Runtime for dbscan_clustering: 0.223565 seconds
Runtime for dbscan_clustering: 0.191605 seconds
Runtime for dbscan_clustering: 0.143635 seconds
Runtime for dbscan_clustering: 0.159599 seconds
Runtime for dbscan_clustering: 0.146052 seconds
Runtime for dbscan_clustering: 0.159258 seconds
Runtime for dbscan_clustering: 0.169363 seconds
Runtime for dbscan_clustering: 0.190180 seconds
Runtime for dbscan_clustering: 0.174931 seconds
Runtime for dbscan_clustering: 0.174240 seconds
Runtime for dbscan_clustering: 0.223315 seconds


...

,Year Range,eps,min_samples,Runtime (s),Clusters,Points per Cluster,Noise Points,Davies-Bouldin,Calinski-Harabasz,Weighted Mean ANOVA F
0,2019-2022,0.020,10,0.033,20,"{0: 71, 1: 15, 2: 17, 3: 10, 4: 36, 5: 15, 6: ...",3262,None,None,NaN
1,2019-2022,0.020,15,0.023,3,"{0: 49, 1: 32, -1: 3528}",3528,None,None,NaN
2,2019-2022,0.020,25,0.037,1,{-1: 3609},3609,None,None,NaN
3,2019-2022,0.020,30,0.031,1,{-1: 3609},3609,None,None,0.0
4,2019-2022,0.010,10,0.015,1,{-1: 3609},3609,None,None,NaN
5,2019-2022,0.010,15,0.020,1,{-1: 3609},3609,None,None,NaN
6,2019-2022,0.010,25,0.019,1,{-1: 3609},3609,None,None,NaN
7,2019-2022,0.010,30,0.019,1,{-1: 3609},3609,None,None,0.0
8,2019-2022,0.015,10,0.032,6,"{0: 20, 1: 21, 2: 10, 3: 11, 4: 10, -1: 3537}",3537,None,None,NaN
9,2019-2022,0.015,15,0.021,1,{-1: 3609},3609,None,None,NaN



Running DBSCAN for Year Range: 2020-2023
Runtime for dbscan_clustering: 0.094976 seconds
Runtime for dbscan_clustering: 0.080393 seconds
Runtime for dbscan_clustering: 0.073501 seconds
Runtime for dbscan_clustering: 0.080064 seconds
Runtime for dbscan_clustering: 0.063390 seconds
Runtime for dbscan_clustering: 0.063852 seconds
Runtime for dbscan_clustering: 0.060745 seconds
Runtime for dbscan_clustering: 0.055810 seconds
Runtime for dbscan_clustering: 0.047822 seconds
Runtime for dbscan_clustering: 0.047640 seconds
Runtime for dbscan_clustering: 0.062133 seconds
Runtime for dbscan_clustering: 0.082419 seconds
Runtime for dbscan_clustering: 0.072542 seconds


c:\Users\fionu\anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:108: RuntimeWarning: invalid value encountered in divide
  msb = ssbn / float(dfbn)
c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:43: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=f_vals, y=features, palette="crest", legend = False)


Best DBSCAN parameters based on ANOVA F-value:
Year Range                2020-2023
eps                            0.02
min_samples                      30
Runtime (s)                   0.080
Clusters                          1
Points per Cluster       {-1: 4372}
Noise Points                   4372
Davies-Bouldin                 None
Calinski-Harabasz              None
Weighted Mean ANOVA F           0.0
Name: 3, dtype: object


,Year Range,eps,min_samples,Runtime (s),Clusters,Points per Cluster,Noise Points,Davies-Bouldin,Calinski-Harabasz,Weighted Mean ANOVA F
0,2020-2023,0.020,10,0.095,26,"{0: 10, 1: 23, 2: 18, 3: 60, 4: 17, 5: 28, 6: ...",3884,None,None,NaN
1,2020-2023,0.020,15,0.080,5,"{0: 15, 1: 53, 2: 15, 3: 33, -1: 4256}",4256,None,None,NaN
2,2020-2023,0.020,25,0.074,1,{-1: 4372},4372,None,None,NaN
3,2020-2023,0.020,30,0.080,1,{-1: 4372},4372,None,None,0.0
4,2020-2023,0.010,10,0.063,1,{-1: 4372},4372,None,None,NaN
5,2020-2023,0.010,15,0.064,1,{-1: 4372},4372,None,None,NaN
6,2020-2023,0.010,25,0.061,1,{-1: 4372},4372,None,None,NaN
7,2020-2023,0.010,30,0.056,1,{-1: 4372},4372,None,None,0.0
8,2020-2023,0.015,10,0.048,8,"{0: 11, 1: 24, 2: 11, 3: 10, 4: 17, 5: 11, 6: ...",4274,None,None,NaN
9,2020-2023,0.015,15,0.048,1,{-1: 4372},4372,None,None,NaN


# DBSCAN
The features used by the clustering are determined by applying pca on the raw dataset.   
After the clustering, we reapply the PCA and plot the clusters in the PC1 / PC2 plane and make the pairplot with the clustered data. 

In [5]:
import cluster_data_pca
from cluster_data_pca import run_clustering, normalize_data, unnormalize
import numpy as np
import pandas as pd
import os
from kmeans import k_means
from cluster_plotter import ClusterPlotter
from clustering_utils_pca import ClusterData
import cluster_plotter
from DBSCAN import dbscan_clustering
import high_dim_analysis
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

standard_year_ranges = {
    f"{start}-{start + 3}": np.arange(start, start + 4)
    for start in [2002, 2006, 2010, 2014, 2019]
}
standard_year_ranges["2019-2023"] = np.arange(2019, 2024)
running_ranges = cluster_data_pca.generate_running_year_ranges(2002, 2023, 4)

binned_data = cluster_data_pca.bin_data_for_clustering(running_ranges, print_res=False)

results_per_year_range = {}

plot_dir = r"Images\\dbscan_tests_running_bins_pcabased_pcaevaluated"
os.makedirs(plot_dir, exist_ok=True)
cluster_plotter.clear_directory(plot_dir)

for cluster_data, year_range in binned_data:
    print(f"\nRunning DBSCAN for Year Range: {year_range}")
    data_array = np.array([
        cluster_data.ecc, cluster_data.mag_obj, cluster_data.sem_maj,
        cluster_data.diameter, cluster_data.inc, cluster_data.raan
    ]).T
    feature_names = ["Eccentricity e", "Object magnitude [mag]", "Semi major axis [km]", "Diameter [m]", "Inclination [°]", "RAAN [°]"]

    normalized_data, data_min, data_max = normalize_data(data_array)

    eps_values = [0.02, 0.01, 0.015]
    min_samples_values = [10, 15, 25, 30]

    results = []

    for eps in eps_values:
        for min_samples in min_samples_values:
            result_dbscan, time_dbscan, n_clusters_dbscan, points_per_cluster_dbscan, metrics_dbscan = run_clustering(
                dbscan_clustering, "DBSCAN", normalized_data, data_min, data_max, eps=eps, min_samples=min_samples
            )

            unnormalized_data, _ = unnormalize(result_dbscan.data, None, data_min, data_max)
            noise_points = np.sum(result_dbscan.labels == -1)

            results.append({
                "Year Range": year_range,
                "eps": eps,
                "min_samples": min_samples,
                "Runtime (s)": f"{time_dbscan:.3f}",
                "Clusters": n_clusters_dbscan,
                "Points per Cluster": points_per_cluster_dbscan,
                "Noise Points": noise_points,
                "Davies-Bouldin": f"{metrics_dbscan[0]:.3f}" if metrics_dbscan and isinstance(metrics_dbscan[0], (int, float)) else None,
                "Calinski-Harabasz": f"{metrics_dbscan[1]:.3f}" if metrics_dbscan and isinstance(metrics_dbscan[1], (int, float)) else None
            })

            labels = result_dbscan.labels

            high_dim_analysis.plot_correlation_heatmap(
                cluster_data,
                output_folder=plot_dir,
                filename=f"corr_{year_range}_eps{eps}_minsamples_{min_samples}.png"
            )
            high_dim_analysis.plot_mutual_information(
                cluster_data,
                labels,
                output_folder=plot_dir,
                filename=f"mi_{year_range}_eps{eps}_minsamples_{min_samples}.png"
            )

            from sklearn.feature_selection import f_classif
            X = data_array
            valid = labels != -1
            if np.any(valid) and len(np.unique(labels[valid])) > 1:
                f_vals, _ = f_classif(X[valid], labels[valid])
                weights = np.var(X[valid], axis=0)
                weighted_mean_f = np.average(f_vals, weights=weights)
            else:
                weighted_mean_f = 0.0

            results[-1]["Weighted Mean ANOVA F"] = f"{weighted_mean_f:.3f}"

    df = pd.DataFrame(results)
    df["Weighted Mean ANOVA F"] = pd.to_numeric(df["Weighted Mean ANOVA F"], errors="coerce")
    df_sorted = df.sort_values(by="Weighted Mean ANOVA F", ascending=False)
    best_params = df_sorted.iloc[0]
    best_eps = best_params["eps"]
    best_min_samples = best_params["min_samples"]

    best_result, _, _, _, _ = run_clustering(
        dbscan_clustering, "DBSCAN", normalized_data, data_min, data_max,
        eps=best_eps, min_samples=best_min_samples
    )
    best_labels = best_result.labels

    df_plot = pd.DataFrame(data_array, columns=feature_names)
    df_plot['cluster'] = best_labels.astype(str)

    pairplot = sns.pairplot(df_plot, hue='cluster', diag_kind='kde', plot_kws={'alpha':0.6, 's':40})
    pairplot.fig.suptitle(f"DBSCAN Clusters for Year Range {year_range}", y=1.02)
    pairplot.savefig(os.path.join(plot_dir, f"pairplot_{year_range}_eps{best_eps}_minsamples_{best_min_samples}.png"))
    plt.close(pairplot.fig)

    high_dim_analysis.plot_anova_f_values(
        cluster_data,
        best_labels,
        output_folder=plot_dir,
        filename=f"anova_best_{year_range}_eps{best_eps}_minsamples_{best_min_samples}.png",
        title=f"ANOVA F-values (Best DBSCAN Parameters), eps = {float(best_eps)}, minsamples = {float(best_min_samples)}"
    )

    # PCA colored by clusters
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data_array)
    pca = PCA(n_components=2)
    components = pca.fit_transform(scaled_data)

    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(components[:, 0], components[:, 1], c=best_labels, cmap='tab10', s=10, alpha=0.7)
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.title(f'PCA Scatter Plot (DBSCAN Clusters) {year_range}')
    plt.grid(True, linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.savefig(os.path.join(plot_dir, f"pca_clusters_{year_range}_eps{best_eps}_minsamples_{best_min_samples}.png"))
    plt.close()

    print("Best DBSCAN parameters based on ANOVA F-value:")
    print(best_params)
    display(df)


Running DBSCAN for Year Range: 2002-2005
Runtime for dbscan_clustering: 0.083050 seconds
Runtime for dbscan_clustering: 0.068838 seconds
Runtime for dbscan_clustering: 0.079146 seconds
Runtime for dbscan_clustering: 0.086649 seconds
Runtime for dbscan_clustering: 0.047804 seconds
Runtime for dbscan_clustering: 0.050260 seconds
Runtime for dbscan_clustering: 0.050540 seconds
Runtime for dbscan_clustering: 0.066400 seconds
Runtime for dbscan_clustering: 0.059345 seconds
Runtime for dbscan_clustering: 0.063897 seconds
Runtime for dbscan_clustering: 0.051811 seconds
Runtime for dbscan_clustering: 0.071604 seconds
Runtime for dbscan_clustering: 0.066436 seconds


c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:43: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=f_vals, y=features, palette="crest", legend = False)


Best DBSCAN parameters based on ANOVA F-value:
Year Range                                                       2002-2005
eps                                                                   0.02
min_samples                                                             15
Runtime (s)                                                          0.069
Clusters                                                                15
Points per Cluster       {0: 15, 1: 21, 2: 210, 3: 717, 4: 13, 5: 25, 6...
Noise Points                                                          6625
Davies-Bouldin                                                        None
Calinski-Harabasz                                                     None
Weighted Mean ANOVA F                                            11163.204
Name: 1, dtype: object


,Year Range,eps,min_samples,Runtime (s),Clusters,Points per Cluster,Noise Points,Davies-Bouldin,Calinski-Harabasz,Weighted Mean ANOVA F
0,2002-2005,0.020,10,0.083,28,"{0: 21, 1: 28, 2: 37, 3: 1009, 4: 17, 5: 141, ...",6159,None,None,7029.527
1,2002-2005,0.020,15,0.069,15,"{0: 15, 1: 21, 2: 210, 3: 717, 4: 13, 5: 25, 6...",6625,None,None,11163.204
2,2002-2005,0.020,25,0.079,6,"{0: 557, 1: 101, 2: 66, 3: 64, 4: 40, -1: 7209}",7209,None,None,102.785
3,2002-2005,0.020,30,0.087,6,"{0: 122, 1: 314, 2: 96, 3: 75, 4: 46, -1: 7384}",7384,None,None,112.751
4,2002-2005,0.010,10,0.048,3,"{0: 33, 1: 22, -1: 7982}",7982,None,None,4.495
5,2002-2005,0.010,15,0.050,2,"{0: 31, -1: 8006}",8006,None,None,0.000
6,2002-2005,0.010,25,0.051,1,{-1: 8037},8037,None,None,0.000
7,2002-2005,0.010,30,0.066,1,{-1: 8037},8037,None,None,0.000
8,2002-2005,0.015,10,0.059,25,"{0: 11, 1: 16, 2: 292, 3: 94, 4: 44, 5: 80, 6:...",7115,None,None,9797.821
9,2002-2005,0.015,15,0.064,8,"{0: 75, 1: 232, 2: 71, 3: 15, 4: 17, 5: 16, 6:...",7587,None,None,5.811



Running DBSCAN for Year Range: 2003-2006
Runtime for dbscan_clustering: 0.117733 seconds
Runtime for dbscan_clustering: 0.080259 seconds
Runtime for dbscan_clustering: 0.100119 seconds
Runtime for dbscan_clustering: 0.100202 seconds
Runtime for dbscan_clustering: 0.066368 seconds
Runtime for dbscan_clustering: 0.071976 seconds
Runtime for dbscan_clustering: 0.066837 seconds
Runtime for dbscan_clustering: 0.049731 seconds
Runtime for dbscan_clustering: 0.083028 seconds
Runtime for dbscan_clustering: 0.061517 seconds
Runtime for dbscan_clustering: 0.076121 seconds
Runtime for dbscan_clustering: 0.066893 seconds
Runtime for dbscan_clustering: 0.099800 seconds


...

,Year Range,eps,min_samples,Runtime (s),Clusters,Points per Cluster,Noise Points,Davies-Bouldin,Calinski-Harabasz,Weighted Mean ANOVA F
0,2019-2022,0.020,10,0.023,20,"{0: 71, 1: 15, 2: 17, 3: 10, 4: 36, 5: 15, 6: ...",3262,None,None,2684.112
1,2019-2022,0.020,15,0.027,3,"{0: 49, 1: 32, -1: 3528}",3528,None,None,607.808
2,2019-2022,0.020,25,0.026,1,{-1: 3609},3609,None,None,0.000
3,2019-2022,0.020,30,0.030,1,{-1: 3609},3609,None,None,0.000
4,2019-2022,0.010,10,0.020,1,{-1: 3609},3609,None,None,0.000
5,2019-2022,0.010,15,0.028,1,{-1: 3609},3609,None,None,0.000
6,2019-2022,0.010,25,0.032,1,{-1: 3609},3609,None,None,0.000
7,2019-2022,0.010,30,0.024,1,{-1: 3609},3609,None,None,0.000
8,2019-2022,0.015,10,0.018,6,"{0: 20, 1: 21, 2: 10, 3: 11, 4: 10, -1: 3537}",3537,None,None,470.541
9,2019-2022,0.015,15,0.018,1,{-1: 3609},3609,None,None,0.000



Running DBSCAN for Year Range: 2020-2023
Runtime for dbscan_clustering: 0.035340 seconds
Runtime for dbscan_clustering: 0.033391 seconds
Runtime for dbscan_clustering: 0.033021 seconds
Runtime for dbscan_clustering: 0.033222 seconds
Runtime for dbscan_clustering: 0.026034 seconds
Runtime for dbscan_clustering: 0.027105 seconds
Runtime for dbscan_clustering: 0.028706 seconds
Runtime for dbscan_clustering: 0.020517 seconds
Runtime for dbscan_clustering: 0.051768 seconds
Runtime for dbscan_clustering: 0.033111 seconds
Runtime for dbscan_clustering: 0.031460 seconds
Runtime for dbscan_clustering: 0.033531 seconds
Runtime for dbscan_clustering: 0.034890 seconds


c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:43: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=f_vals, y=features, palette="crest", legend = False)


Best DBSCAN parameters based on ANOVA F-value:
Year Range                                                       2020-2023
eps                                                                   0.02
min_samples                                                             10
Runtime (s)                                                          0.035
Clusters                                                                26
Points per Cluster       {0: 10, 1: 23, 2: 18, 3: 60, 4: 17, 5: 28, 6: ...
Noise Points                                                          3884
Davies-Bouldin                                                        None
Calinski-Harabasz                                                     None
Weighted Mean ANOVA F                                             2954.547
Name: 0, dtype: object


,Year Range,eps,min_samples,Runtime (s),Clusters,Points per Cluster,Noise Points,Davies-Bouldin,Calinski-Harabasz,Weighted Mean ANOVA F
0,2020-2023,0.020,10,0.035,26,"{0: 10, 1: 23, 2: 18, 3: 60, 4: 17, 5: 28, 6: ...",3884,None,None,2954.547
1,2020-2023,0.020,15,0.033,5,"{0: 15, 1: 53, 2: 15, 3: 33, -1: 4256}",4256,None,None,85.899
2,2020-2023,0.020,25,0.033,1,{-1: 4372},4372,None,None,0.000
3,2020-2023,0.020,30,0.033,1,{-1: 4372},4372,None,None,0.000
4,2020-2023,0.010,10,0.026,1,{-1: 4372},4372,None,None,0.000
5,2020-2023,0.010,15,0.027,1,{-1: 4372},4372,None,None,0.000
6,2020-2023,0.010,25,0.029,1,{-1: 4372},4372,None,None,0.000
7,2020-2023,0.010,30,0.021,1,{-1: 4372},4372,None,None,0.000
8,2020-2023,0.015,10,0.052,8,"{0: 11, 1: 24, 2: 11, 3: 10, 4: 17, 5: 11, 6: ...",4274,None,None,415.989
9,2020-2023,0.015,15,0.033,1,{-1: 4372},4372,None,None,0.000
